In [1]:
%load_ext autoreload
%autoreload 2

# Non-adiabatic dynamics 
This tutorial shows how to run non-adiabatic dynamics with a trained model using the [Zhu-Nakamura surface hopping method](https://pubs.rsc.org/en/content/articlelanding/2014/cp/c4cp03498h).

First let's import dependencies:


In [2]:
import sys
import os

# so that NFF is in your path
sys.path.insert(0, "..")

import json
import numpy as np
from nff.md.zhu_nakamura.dynamics import CombinedZhuNakamura
from ase.io.trajectory import Trajectory
import nglview as nv
from nff.md.utils import csv_read
from ase import Atoms
from nff.md.zhu_nakamura.run_zn import coords_to_xyz, make_dataset, make_trj




Now we'll find a trained model. The trained azobenzene models can be found in `NeuralForceField/models/azo_derivatives`. The sub-folders are for diabatic and adiabatic models, trained either with the full set of geometries, or with 40 species held out. There are also three models trained with different splits and different initialization from random seeds:

In [3]:
print(os.listdir('../models/azo_derivatives'))
print(os.listdir('../models/azo_derivatives/all_diabatic'))

['all_diabatic', 'holdout_diabatic', 'holdout_adiabatic']
['seed_0', 'seed_2', 'seed_1']


We'll use the diabatic model trained on all species, with seed 0: `../models/azo_derivatives/all_diabatic/seed_0`.

## Zhu-Nakamura

The script for ZN surface hopping is `NeuralForceField/nff/md/zhu_nakamura/run_zn.py`. If you run the script and supply the path of a JSON parameter file, it will do the rest for you. Here we'll go through some parameters to give, and show a little of what goes on behind the scenes.

We'll have to define `ground_params`, `zhu_params`, `combined_params`, and `all_params`. The first is for parameters in the ground state MD simulation, the second for ZN surface hopping, and the third is for batching. The fourth is for some remaining parameters, which we'll explain below.

Let's define `ground_params`:

In [4]:
ground_params = {'ttime': 50, # tau = ttime * dt is the relaxation time
                 'logfile': 'ground.log', # log file for ground state MD
                 'max_time': 200, # total time in fs
                 'savefile': 'ground.trj', # output file with saved geometries
                 'timestep': 0.5, # dt in fs
                 'equil_time': 100, # ignore this amount of time (fs) when sampling
                                    # geoms for NAMD 
                 'thermostat': 'nosehoover', # use the Nose Hoover thermostat
                 'loginterval': 10, # log the energy and save geoms every 10 steps
                 'temperature': 300, # temperature in Kelvin
                 'cutoff': 5.0, # neighbor list cutoff in Angstrom 
                 'cutoff_skin': 2.0, # extra distance added to cutoff when updating
                                     # neighbor list, to account for atoms coming into
                                     # the 5 A sphere between updates 
                }

Now let's do `zhu_params` and `batched_params`:

In [5]:
zhu_params = {'log_file': 'trj.log', # log file for NAMD
              'max_time': 200, # total time in fs
              'out_file': 'trj.csv', # the i^th trajectory gets a csv called `trj_{i}.csv` with all its info
              'timestep': 0.5, # dt in fs
              'num_states': 2, # number of adiabatic states
              'max_gap_hop': 0.021673306772908366, # don't hop if the gap is over 0.0217 au (0.59 eV)
              'save_period': 5, # save every 5 steps
              'initial_surf': 1, # start on the first excited state
              'initial_time': 0.0 # start at t=0
             }

batched_params = {'cutoff': 5.0, # neighbor list cutoff in Angstrom 
                  'cutoff_skin': 2.0, # extra distance added to cutoff when updating
                                      # neighbor list, to account for atoms coming into
                                      # the 5 A sphere between updates 
                  'device': 1, # Use GPU 1 (set to 'cpu' if you don't have a GPU)
                  'num_trj': 10, # Number of trajectories
                  'batch_size': 5, # Number of trajectories to batch together in one so that calculations
                                   # can be done in parallel
                  'nbr_update_period': 10, # update the neighbor list every 10 steps
                  
                 }

Lastly we'll define `all_params`, which has the starting coordinates and the model path:

In [6]:
with open('data/azo_coords.json', 'r') as f:
    coords = json.load(f)

all_params = {"coords": coords, # starting geometry of the molecule
              'model_path': '../models/azo_derivatives/all_diabatic/seed_0', # path to saved model
              "zhu_params": zhu_params,
              "ground_params": ground_params,
              "batched_params": batched_params
             }


When we run the script from the command line, it parses these three dictionaries from a file and makes an instance of `CombinedZhuNakamura`, like this:



In [7]:

coords = all_params["coords"]
nxyz = [coords_to_xyz(coords)]


print('loading models')

dataset = make_dataset(nxyz=nxyz, 
                       all_params=all_params)

print('running ground state + Zhu-Nakamura dynamics')

zn = make_trj(all_params=all_params,
              dataset=dataset)



loading models
running ground state + Zhu-Nakamura dynamics


/home/saxelrod/miniconda3/envs/nff/lib/python3.7/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


First we made the xyz of the structure from the dictionary of coordinates. Then we turned it into an NFF dataset and used that to make an instance of `CombinedZhuNakamura`.

For an example of how you would use this script in practice, check out `data/zn_info.json`. If you run
```bash
conda activate nff
python ../nff/md/zhu_nakamura/run_zn.py data/zn_info.json
```
then you should be able to peform neural ZN in one line. Note that in `zn_info.json`, the `all_params` part of the dictionary is its body, i.e. everything that doesn't have the key `ground_params`, `zhu_params`, or `batched_params`.


Now we can run it!

In [8]:
zn.run()


../nff/nn/modules/diabat.py:140: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/aten/src/ATen/native/BatchLinearAlgebra.cpp:2500.)
  ad_energies, u = torch.symeig(d_mat, True)


Completed step 0
Completed step 10
Completed step 20
Completed step 30
Completed step 40
Completed step 50
Completed step 60
Completed step 70
Completed step 80
Completed step 90
Completed step 100
Completed step 110
Completed step 120
Completed step 130
Completed step 140
Completed step 150
Completed step 160
Completed step 170
Completed step 180
Completed step 190
Completed step 200
Completed step 210
Completed step 220
Completed step 230
Completed step 240
Completed step 250
Completed step 260
Completed step 270
Completed step 280
Completed step 290
Completed step 300
Completed step 310
Completed step 320
Completed step 330
Completed step 340
Completed step 350
Completed step 360
Completed step 370
Completed step 380
Completed step 390
Completed step 400
Neural ZN terminated normally.


We can view the ground-state log file:

In [9]:
with open('ground.log', 'r') as f:
    ground_log = f.read()
print(ground_log)

Time[ps]      Etot[eV]     Epot[eV]     Ekin[eV]    T[K]
0.0000           2.1950      -0.8128       3.0077   684.4
0.0050           2.1642       1.0735       1.0907   248.2
0.0100           2.1265       0.4109       1.7156   390.4
0.0150           2.0768       0.9340       1.1428   260.0
0.0200           2.0272       0.4267       1.6005   364.2
0.0250           1.9668       0.7417       1.2251   278.8
0.0300           1.9003       0.3549       1.5454   351.6
0.0350           1.8343       0.8340       1.0004   227.6
0.0400           1.7740       0.3706       1.4035   319.3
0.0450           1.6986       0.4146       1.2840   292.2
0.0500           1.6278       0.4870       1.1408   259.6
0.0550           1.5603       0.3793       1.1810   268.7
0.0600           1.4958       0.4702       1.0256   233.4
0.0650           1.4402       0.3627       1.0775   245.2
0.0700           1.3939       0.6191       0.7748   176.3
0.0750           1.3623       0.2954       1.0669   242.8
0.0800         

We see that all energies fluctuate, as kinetic energy is being added into the system fo the thermostat. The temperature also varies, and over enough time it will average out to 300 K. 

To get the actual geometries, energies, and forces, we can load the trajectory file. And we can visualize it with `nglview`:



In [10]:
trj = Trajectory('ground.trj')
nv.show_asetraj(trj)

NGLWidget(max_frame=40)

Unlike neural Tully, neural ZN saves the trajectories separately from each other. This may be changed in the future, since saving in one file is much easier. In any case we can examine individual trajectories:

In [11]:
with open('trj_4.log', 'r') as f:
    zn_log = f.read()
print(zn_log)

ZHU-NAKAMURA DYNAMICS:  Completed step 1. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 2.9237707687275827 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 11. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 2.6335102532362833 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 21. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 2.0879162426682454 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 31. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 1.4902326004829958 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 40. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 0.8456784093746563 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 50. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 0.3471257561769035 eV
ZHU-NAKAMURA DYNAMICS:  Completed step 61. Currently in state 1.
ZHU-NAKAMURA DYNAMICS:  Relative energies are 0.0, 0.6041592334855956 eV
ZHU-NAKAMURA DYNAMICS:  Completed s

To get the geometries, forces, etc., we can load the trajectory's CSV file:



In [12]:
trj_dics = csv_read('trj_4.csv')

We can turn the xyz's into an ase trajectory and visualize it:


In [13]:
nxyz_list = [np.array(i['nxyz']) for i in trj_dics]
trj = [Atoms(numbers=nxyz[:, 0], positions=nxyz[:, 1:])
      for nxyz in nxyz_list]
nv.show_asetraj(trj)

NGLWidget(max_frame=40)

We can also see some properties. As in neural Tully, everything is given in atomic units, except for coordinates which are given in Angstroms.

In [19]:
trj_dics[0].keys()

dic = trj_dics[0]
print(np.array(dic['energy']).shape) # adiabatic energies
print(np.array(dic['force']).shape) # forces on each state
print(dic['hopping_probability']) # list of dictionaries of ZN a, b, and p parameters between all pairs of
                                  # states if an avoided crossing is encountered
print(dic['in_trj']) # whether this frame is in the trajectory. An avoided crossing is identified as a geom
                     # for which the previous and subsequent geoms had higher gaps. In ZN a hop only occurs
                     # at avoided crossings. If a hop does occur, then the frame right after the avoided crossing
                     # is no longer part of the trajectory, because you move one frame back to the avoided 
                     # crossing, and switch energy surfaces. Therefore, those geoms would have `in_trj=False`,
                     # and every other one has `in_trj=True`
print(np.array(dic['nxyz']).shape) # xyz at this time step
print(np.array(dic['position']).shape) # same as nxyz but without atomic numbers
print(np.array(dic['surf'])) # current surface 
print(dic['time']) # current time (fs)
print(np.array(dic['velocity']).shape) # current velocity 

(2,)
(2, 34, 3)
[]
True
(34, 4)
(34, 3)
1
0.5
(34, 3)
